<a href="https://colab.research.google.com/github/JohnDiognes/analise-perfil-eleitor-vila-velha/blob/main/notebooks/03_An%C3%A1lise_Geogr%C3%A1fica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 03 — Análise Geográfica (Mapas por Bairro)
Perfil do Eleitor — Município de Vila Velha – ES  
Autor: *Diognes Gonçalves dos Santos*  
Projeto: Portfólio Profissional — Área de Dados  

---

## 🎯 Objetivo do Notebook

Este notebook tem como objetivo:

- Integrar as bases de eleitores com a malha geográfica dos bairros de Vila Velha.  
- Criar mapas temáticos (*choropleth maps*) com **Folium**.  
- Visualizar distribuições por:
  - Quantidade de eleitores por bairro
  - Perfil etário
  - Distribuição por gênero
- Preparar camadas geográficas reutilizáveis para dashboards (Power BI, Tableau etc.)

---


1. Importar bibliotecas

In [5]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt


2. Carregar dados

- `base_vila_velha.csv`: dataset consolidado no Notebook 02  
- `BairroXRegiao.csv`: tabela auxiliar  
- `shapefile dos bairros`: será necessário para os mapas  


Carregar base de eleitores

In [19]:
!rm -rf analise-perfil-eleitor-vila-velha
!git clone https://github.com/JohnDiognes/analise-perfil-eleitor-vila-velha.git



Cloning into 'analise-perfil-eleitor-vila-velha'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 172 (delta 76), reused 50 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (172/172), 11.27 MiB | 26.47 MiB/s, done.
Resolving deltas: 100% (76/76), done.
Filtering content: 100% (7/7), 915.57 MiB | 30.88 MiB/s, done.
Encountered 2 file(s) that should have been pointers, but weren't:
	Documents/Projeto_perfil_eleitor_2024_ES/BairroXRegiao.csv
	Documents/Projeto_perfil_eleitor_2024_ES/Local_votacao_Vila_Velha.csv


In [4]:
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# -----------------------------
# Clonar o repositório (somente se não existir)
# -----------------------------
repo_nome = "analise-perfil-eleitor-vila-velha"

if not os.path.exists(repo_nome):
    !git clone https://github.com/JohnDiognes/analise-perfil-eleitor-vila-velha.git

# Caminho da base
caminho_base = f"{repo_nome}/dados/base_vila_velha.csv"

# -----------------------------
# Carregar a base
# -----------------------------
try:
    df_vv = pd.read_csv(caminho_base, encoding="utf-8")
except:
    df_vv = pd.read_csv(caminho_base, encoding="latin1")

df_vv.head()

Cloning into 'analise-perfil-eleitor-vila-velha'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 129 (delta 51), reused 51 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (129/129), 11.20 MiB | 26.18 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Filtering content: 100% (7/7), 915.57 MiB | 67.04 MiB/s, done.
Encountered 2 file(s) that should have been pointers, but weren't:
	Documents/Projeto_perfil_eleitor_2024_ES/BairroXRegiao.csv
	Documents/Projeto_perfil_eleitor_2024_ES/Local_votacao_Vila_Velha.csv


,ano,uf,municipio,cod_municipio,zona,secao,cod_genero,genero,cod_estado_civil,estado_civil,...,interprete de libras,bairro,regiao,cod_local_votacao,local_votacao_nome,local_votacao_endereco,quant_eleitores,quant_biometria,quan_deficiencia,quant_nome_social
0,2024,ES,VILA VELHA,57037,32,266,4,FEMININO,3,CASADO,...,NÃO INFORMADO,PAUL,3,1430,UMEF GRACIANO NEVES,"R. COUTO AGUIRRE, 49",1,1,0,0
1,2024,ES,VILA VELHA,57037,32,951,4,FEMININO,3,CASADO,...,NÃO INFORMADO,ITAPOÃ,1,2070,COLÉGIO CEIC,"RUA JAIME DUARTE NASCIMENTO, 617",1,1,0,0
2,2024,ES,VILA VELHA,57037,32,936,4,FEMININO,3,CASADO,...,NÃO INFORMADO,ITAPOÃ,1,2070,COLÉGIO CEIC,"RUA JAIME DUARTE NASCIMENTO, 617",1,1,1,0
3,2024,ES,VILA VELHA,57037,57,155,4,FEMININO,1,SOLTEIRO,...,NÃO INFORMADO,SÃO TORQUATO,4,1201,UMEF JUIZ JAIRO DE MATTOS PEREIRA,"RUA PASTOR AMBROSINO BARBOSA, S/ N",1,1,0,0
4,2024,ES,VILA VELHA,57037,57,48,2,MASCULINO,3,CASADO,...,NÃO INFORMADO,COBILÂNDIA,4,1341,CEEFMTI PASTOR OLIVEIRA DE ARAUJO- ESCOLA VIVA,"AVENIDA OTÁVIO BORIN, S/N",1,1,0,0


Carregar tabela de bairros e regiões

In [8]:
try:
    bairros = pd.read_csv(f"{repo_nome}/Documents/Projeto_perfil_eleitor_2024_ES/BairroXRegiao.csv", encoding="utf-8")
except UnicodeDecodeError:
    bairros = pd.read_csv(f"{repo_nome}/Documents/Projeto_perfil_eleitor_2024_ES/BairroXRegiao.csv", encoding="latin1")
bairros.head()

,num_regiao,nom_bairro
0,1,Boa Vista I
1,1,Boa Vista II
2,1,Centro
3,1,Coqueiral de Itaparica
4,1,Cristóvão Colombo


Importar shapefile de Vila Velha

colocar o shapefile na pasta dados/geodata/ como:

In [21]:
mapa = gpd.read_file(f"{repo_nome}/dados/geodata/bairros_vila_velha.shp")
mapa.head()

,BAIRRO,CODBAIRRO,DocID,OwnerID,CadType,Level,Color,LineWt,Layer,Handle,...,TxtAttach,TxtDir,LnSpace,SpaceFact,DocName,DocPath,DocType,DocVer,XDList,geometry
0,JABURUNA,18,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,None,None,None,None,None,"POLYGON ((363434.724 7751165.889, 363432.91 77..."
1,SANTA MÔNICA,161,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,None,None,None,None,None,"POLYGON ((363761.592 7748055.804, 363904.666 7..."
2,RESIDENCIAL COQUEIRAL,67,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,None,None,None,None,None,"POLYGON ((364130.917 7748346.266, 364118.952 7..."
3,SOTECO,66,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,None,None,None,None,None,"POLYGON ((363402.978 7748499.386, 363415.998 7..."
4,COQUEIRAL DE ITAPARICA,12,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,None,None,None,None,None,"POLYGON ((363381.341 7747424.748, 363449.897 7..."


🧩 3. Preparar base geográfica
Normalizar nomes de bairros

In [24]:
df_vv['bairro'] = df_vv['bairro'].str.upper().str.strip()
mapa['BAIRRO'] = mapa['BAIRRO'].str.upper().str.strip()

Cruzar base de eleitores + mapa

In [25]:
df_bairro = df_vv.groupby('bairro').size().reset_index(name='qtd_eleitores')
df_geo = mapa.merge(df_bairro, left_on='BAIRRO', right_on='bairro', how='left')
df_geo['qtd_eleitores'] = df_geo['qtd_eleitores'].fillna(0)
df_geo.head()

,BAIRRO,CODBAIRRO,DocID,OwnerID,CadType,Level,Color,LineWt,Layer,Handle,...,LnSpace,SpaceFact,DocName,DocPath,DocType,DocVer,XDList,geometry,bairro,qtd_eleitores
0,JABURUNA,18,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,None,None,None,None,None,"POLYGON ((363434.724 7751165.889, 363432.91 77...",JABURUNA,1617.0
1,SANTA MÔNICA,161,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,None,None,None,None,None,"POLYGON ((363761.592 7748055.804, 363904.666 7...",SANTA MÔNICA,2975.0
2,RESIDENCIAL COQUEIRAL,67,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,None,None,None,None,None,"POLYGON ((364130.917 7748346.266, 364118.952 7...",NaN,0.0
3,SOTECO,66,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,None,None,None,None,None,"POLYGON ((363402.978 7748499.386, 363415.998 7...",SOTECO,6451.0
4,COQUEIRAL DE ITAPARICA,12,NaN,NaN,None,NaN,NaN,NaN,None,None,...,NaN,NaN,None,None,None,None,None,"POLYGON ((363381.341 7747424.748, 363449.897 7...",COQUEIRAL DE ITAPARICA,15066.0


4. Mapa — Eleitores por Bairro

In [27]:
m = folium.Map(location=[-20.3477, -40.2949], zoom_start=12)

folium.Choropleth(
    geo_data=df_geo,
    data=df_geo,
    columns=["BAIRRO", "qtd_eleitores"],
    key_on="feature.properties.BAIRRO",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Quantidade de Eleitores por Bairro"
).add_to(m)

folium.LayerControl().add_to(m)

m

5. Mapa — Percentual de Mulheres por Bairro

In [ ]:
df_genero = df.groupby(["bairro", "genero"]).size().unstack(fill_value=0)
df_genero["pct_mulheres"] = df_genero["FEMININO"] / df_genero.sum(axis=1)

df_geo = df_geo.merge(df_genero["pct_mulheres"], on="bairro", how="left")


In [ ]:
m2 = folium.Map(location=[-20.3477, -40.2949], zoom_start=12)

folium.Choropleth(
    geo_data=df_geo,
    data=df_geo,
    columns=["bairro", "pct_mulheres"],
    key_on="feature.properties.bairro",
    fill_color="PuRd",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Percentual de Mulheres"
).add_to(m2)

m2


6. Mapa — Faixas Etárias

In [ ]:
Criar indicador numérico de idade

In [ ]:
df['idade'] = 2024 - df['ano_nascimento']


In [ ]:
faixas = pd.cut(
    df['idade'],
    bins=[16, 24, 34, 44, 59, 120],
    labels=["16–24", "25–34", "35–44", "45–59", "60+"]
)

df['faixa_etaria'] = faixas


Distribuição por bairro

In [ ]:
df_idade = df.groupby(["bairro", "faixa_etaria"]).size().reset_index(name="total")
df_idade.head()


Exemplo: mapa da faixa etária “60+”

In [ ]:
idosos = df_idade[df_idade["faixa_etaria"] == "60+"]

df_geo_idosos = mapa.merge(idosos, on="bairro", how="left")
df_geo_idosos["total"] = df_geo_idosos["total"].fillna(0)


In [ ]:
m3 = folium.Map(location=[-20.3477, -40.2949], zoom_start=12)

folium.Choropleth(
    geo_data=df_geo_idosos,
    data=df_geo_idosos,
    columns=["bairro", "total"],
    key_on="feature.properties.bairro",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Quantidade de Eleitores 60+ por Bairro"
).add_to(m3)

m3


7. Heatmap — Concentração de Eleitores

In [ ]:
df_latlon = df.dropna(subset=["latitude", "longitude"])


In [ ]:
m4 = folium.Map(location=[-20.3477, -40.2949], zoom_start=12)

HeatMap(
    data=df_latlon[['latitude', 'longitude']].values,
    radius=12,
    blur=15
).add_to(m4)

m4


📌 8. Conclusões do Notebook

O cruzamento das bases permitiu gerar mapas temáticos por bairro.

Identificamos regiões com maior densidade de eleitores.

Foram criados indicadores geográficos que podem ser exportados para dashboards.

Os mapas interativos permitem explorar rapidamente padrões demográficos no município.

9. Exportar base geográfica para uso externo (Power BI, Tableau)

In [ ]:
df_geo.to_file("../dados/geodata/saida_bairros.geojson", driver="GeoJSON")
